In [133]:
import time
import requests
from bs4 import BeautifulSoup
import pymongo
import pandas as pd
import numpy as np

In [16]:
mc = pymongo.MongoClient()  # Connect to the MongoDB server using default settings
db = mc['strain_reviews']  # Use (or create) a database called 'election_predictions'
docs = db['review']  # Use (or create) a collection called 'docs'

In [57]:
docs.count_documents(filter={})

0

In [58]:
docs.drop()

## Here is how i want the scrape to work

    """LOS = [List of Strains]
        for s in LOS:
        url = "https://www.leafly.com/{}/reviews?page=".format(s)
        for i in range(1,100):
            d = get_reviews(url+str(i))
            st, rev = parse_docs(d)
            if len(st) > 0:
               append.(add to your dictionary)
            else:
                break"""

In [19]:
def table_check2(l):
    """This checks to see if a url is already in my database of reviews. 
    Only needs to be run in the get reviews function"""
    for url in l:
        if docs.find_one({'url':url}):
            None
        else:
            docs.insert_one({'url': url,
                 'html': requests.get(url).content
                 })

In [20]:
def get_reviews2(l):
    """Pass in a list of URL's and return them in a mongo db table as a dicitonary with 
    {'url', 'html'} and their corresponding values"""
    r = requests.get(l)
    html = (r.content)
    return html

In [21]:
def get_stars_list(d):
    stars = []
    for key, values in d.items():
        soup = BeautifulSoup(values, 'html.parser')
        tags = soup.select("div.div.stars")
        for t in tags:
            stars.append(t.attrs['style'])
    return stars

In [22]:
def star_int_conv(l):
    flat_stars= []
    star_num = []
    for s in l:
        star = int((s[6:].split(';')[0]).strip('px'))
        star_num.append((star/22))
    return star_num

In [367]:
def list_o_strains(i):
    LOS = []
    r = requests.get(i)
    soup2 = BeautifulSoup(r.content, 'html.parser')
    strains = soup2.find_all('a', class_="ga_Explore_Strain_Tile")
    for s in strains:
        LOS.append(str(s.attrs['href'])[1:])
    return LOS

In [24]:
def parse_docs2(d):
    """Parse the HTML docs that we have stored in a dictionary, return as a list.
    Also scrape and parse star rating for each review """
    strain_text= []
    star_rate = []
    user_name = []
    soup = BeautifulSoup(d, 'html.parser')
    revs = soup.find_all('p',class_='strain-review__text') 
    for r in revs:
        strain_text.append(r.text)
    tags = soup.select("div.div.stars")
    star = []
    for t in tags:
        star.append(t.attrs['style'])
    star_rate.append(star_int_conv(star))
    users = soup.find_all('div', class_='strain-review__title')
    for u in users:
        temp = u.find('h2')
        user_name.append(temp.text)
    return star_rate, strain_text, user_name

In [25]:
#Test works better, is updated!
def scraper_dummy1(los):
    los_urls = []
    rev_urls = []
    for s in los:
        url = "https://www.leafly.com{}/reviews?page=".format(s)
        print(url)
        for i in range(1,100):
            rev_urls=url+str(i)
            d = get_reviews2(rev_urls)
            star, reviews, users = parse_docs2(d)
            print(users)
            if len(star[0]) == 0:
                break

In [413]:
def scraper_dummy_test(los):
    los_urls = []
    rev_urls = []
    user_list_of_lists = []
    star_list_of_lists = []
    reviews_list_o_lists = []
    #grab URL landing pages for reviews on each strain.
    for s in los:
        url = "https://www.leafly.com/{}/reviews?page=".format(s)
        strain_star_list = []
        strain_user_list = []
        strain_rev_list = []
        #get all reviews, stars, and users for each strain in list of strains
        for i in range(1,100):
            rev_urls=url+str(i)
            d = get_reviews2(rev_urls)
            star, reviews, users = parse_docs2(d)
            #return stars, strains, reviews, in one list, not list of lists for each page scraped. 
            #star_unlisted = 
            strain_star_list.extend([x for x in star[0]])
            strain_user_list.extend(users)
            strain_rev_list.extend(reviews)
            if len(star[0]) == 0:
                break
        #these are long single lists, not list of lists
        star_list_of_lists.append(strain_star_list)
        user_list_of_lists.append(strain_user_list)
        reviews_list_o_lists.append(strain_rev_list)
        #make into dictionaries 
        for s in los:
            star_ds = []
            rev_ds = []
            for i in range(len(star_list_of_lists)):
                user_revs =  dict(list(zip(user_list_of_lists[i], reviews_list_o_lists[i])))
                rev_ds.append(user_revs)
                user_stars = dict(list(zip(user_list_of_lists[i], star_list_of_lists[i])))
                star_ds.append(user_stars)
    #create Pandas DF of each dictionary as well.
    star_dfs = []
    rev_dfs = []
    for s in star_ds:
        star_dfs.append(pd.DataFrame.from_dict(s, orient='index')) 
    for r in rev_ds: 
        rev_dfs.append(pd.DataFrame.from_dict(r, orient ='index'))
    #create large dataframe
    star_df = pd.concat(star_dfs, axis=1, sort=True)
    rev_df = pd.concat(rev_dfs, axis=1, sort=True)
    rev_df.columns = los
    star_df.columns = los
    return star_df, rev_df

In [414]:
test_strains = list_o_strains('https://www.leafly.com/explore')

In [415]:
test = test_strains[:3]
test

['hybrid/blue-dream', 'sativa/sour-diesel', 'hybrid/gsc']

In [416]:
stars, reviews = scraper_dummy_test(test)

In [419]:
#stars.columns = test
stars

,hybrid/blue-dream,sativa/sour-diesel,hybrid/gsc
1012crew,5.0,NaN,NaN
1033InProgress,NaN,4.0,NaN
11scorpio11,5.0,NaN,NaN
12345goog,NaN,NaN,2.0
123Buddy,5.0,NaN,NaN
1kushking,NaN,NaN,5.0
1ofthisother,5.0,NaN,5.0
1olegrumpy1,5.0,NaN,NaN
1taste,4.0,5.0,NaN
215CutReviews629,NaN,NaN,4.0


In [237]:
pd.concat(reviews, axis=1)

/Users/jordanweil/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,0,0,0
13Gawd,"""Very good for my anxiety and doing more socia...",NaN,"""Very good for my anxiety and doing more socia...",NaN
152gjeng,NaN,"""This strain has always been extremely harsh t...",NaN,"""This strain has always been extremely harsh t..."
1taste,NaN,"""Honestly this is not a favourite of mine. Yes...",NaN,"""Honestly this is not a favourite of mine. Yes..."
206SEA,"""insane. glued to the couch """,NaN,"""insane. glued to the couch """,NaN
419plus1,NaN,"""Energetic which is great, but paranoid.""",NaN,"""Energetic which is great, but paranoid."""
420life,"""Great gorilla glue #4 strain in volcano vapo...",NaN,"""Great gorilla glue #4 strain in volcano vapo...",NaN
420reyes420,"""Gorilla Glue number 4 is definitely a chillax...",NaN,"""Gorilla Glue number 4 is definitely a chillax...",NaN
561.welly,"""I loved Gorilla Glue. Its high was super mell...",NaN,"""I loved Gorilla Glue. Its high was super mell...",NaN
8CoreXeon,NaN,"""Got mine from Route 66 Clones + Cannabis in T...",NaN,"""Got mine from Route 66 Clones + Cannabis in T..."
8characters,"""Vaping this out of my Pax Era is beautiful""",NaN,"""Vaping this out of my Pax Era is beautiful""",NaN


In [191]:
user_stars_gg = dict(list(zip(users[0], stars[0])))

In [192]:
user_revs_gg = dict(list(zip(users[0], reviews[0])))

In [193]:
user_revs_gg

{'chronicallymedicated': '"Initial impression: My bud-tender said in the usual tone "what\'ll it be?" I chirped back, "surprise me, something new from the cannaseur shelf\' please". So I was handed a newer strain aptly named "Mother\'s Milk. The other one was Gorilla Glue #4 (GG)" (reviewed in this post). Needless to say, I was pretty excited about both of these. I sampled this one in a pre-rolled cone form, also rolled with "Raw Cones" thanks to..."',
 'MZ_PHD_in_420': '"SOON TO BE TOP !0 STRAIN OF ALL TIME!!!I have 40+ years experience growing and using cannabis, a masters degree in agri/horticulture from MSU and will break each review down as follows from my own personal experience with seeds or clones obtained directly from the original breeders. I received my clone directly from the person that entered it in the 2014 Michigan High Times Cannabis Cup, where it won for the second t..."',
 'gatorhunts420': '"Shhh don\'t tell anybody but I rolled my best shit from my last harvest of t

In [190]:
user_revs.keys()

dict_keys(['hi2', 'MurderMyLove', 'JJMaxwell', 'greeneyeganja.girl', 'GreenPie', 'Ramoburo', 'kantheist', 'iamcherryface', 'GanjaGoddess410', 'ChemicalHazex', 'BobCantwell', 'BadBehavior89', 'KindGodess420', 'willir97', 'somacolorado', 'CanadaKush', 'John18', 'Pikarya', 'TSmoove', 'whitewidow81', 'allnaturalfitness', 'Flakzter', 'xSarahcate', 'StonerStig', 'bobross559', 'srubek', 'ImaFlyboy', 'ArthritisPain', 'Lumo', 'Reighly', 'BakinKush', 'laj821', 'hurdyhurdy', 'Dryland2012', 'highmonkey', 'kalamardesk', 'StonedHousewife', 'HudsonCole', 'RavenBanes', 'Kayley14', 'cmgunder', 'Spanish420', 'bdriskell99', 'DiscordedDiscord', 'inthewild', 'Deevaa', 'vito.genovese.7568', 'Aaro', 'artyfartyblah', 'googleymoo', 'weeddude1234', 'Velocity666', 'LemonKatt', 'CANNIBALCORPSE666', 'Anonbrra', 'Trinli66', 'Paperplaneprincess09', 'MrsRachBall13', 'LsdInMyWee', 'Apil', 'blommen', 'tank7', 'Sloppysundance', '1taste', 'drealii', 'Bartndk', 'blackcoffeekush', 'JuJumanski', 'TeamScienceOttawa', 'Harryp

In [194]:
GG4_df = pd.DataFrame.from_dict(user_stars_gg, orient='index', columns=test_strains[6:7])

In [195]:
WhiteWid_df = pd.DataFrame.from_dict(user_stars, orient='index', columns=test_strains[6:7])

In [199]:
concat_df = [GG4_df, WhiteWid_df]

In [200]:
pd.concat(concat_df, axis=1)

/Users/jordanweil/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,/hybrid/original-glue,/hybrid/original-glue
13Gawd,5.0,NaN
152gjeng,NaN,3.0
1taste,NaN,3.0
206SEA,5.0,NaN
419plus1,NaN,4.0
420life,5.0,NaN
420reyes420,5.0,NaN
561.welly,5.0,NaN
8CoreXeon,NaN,4.0
8characters,5.0,NaN
